In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import time
import re
import networkx as nx
from operator import itemgetter
import warnings
from tqdm.notebook import tqdm
import glob
warnings.filterwarnings('ignore')

# Indice

- [Load Data](#load)
- [General info](#gi)
- [URLs](#urls)
- [Credibility](#cred)
- [Coordinated account](#coord)

### Functions

In [2]:
def get_list_by_keyword(subject, keyword):
    list_by_key = []
    indices = [i for i, x in enumerate(subject.split()) if keyword in x]
    if len(indices) == 0:
        return None
    list_by_key = [None] * len(indices)
    for idx in range(len(indices)):
        list_by_key[idx] = subject.split()[int(indices[idx]+1)].replace("'", "").replace(",", "")
    return list_by_key

### Load data
<a id="load"/>

In [3]:
path = 'C:/Users/gianl/Desktop/Gi/Supsi/MartaLauraGiada/Marta/Tweets/'
files = glob.glob(path + "202*")
files

['C:/Users/gianl/Desktop/Gi/Supsi/MartaLauraGiada/Marta/Tweets\\2020-01.parquet',
 'C:/Users/gianl/Desktop/Gi/Supsi/MartaLauraGiada/Marta/Tweets\\2020-02-1.parquet',
 'C:/Users/gianl/Desktop/Gi/Supsi/MartaLauraGiada/Marta/Tweets\\2020-02-2.parquet',
 'C:/Users/gianl/Desktop/Gi/Supsi/MartaLauraGiada/Marta/Tweets\\2020-03-1.parquet',
 'C:/Users/gianl/Desktop/Gi/Supsi/MartaLauraGiada/Marta/Tweets\\2020-03-2.parquet',
 'C:/Users/gianl/Desktop/Gi/Supsi/MartaLauraGiada/Marta/Tweets\\2020-04-1.parquet',
 'C:/Users/gianl/Desktop/Gi/Supsi/MartaLauraGiada/Marta/Tweets\\2020-04-2.parquet',
 'C:/Users/gianl/Desktop/Gi/Supsi/MartaLauraGiada/Marta/Tweets\\2020-05-1.parquet',
 'C:/Users/gianl/Desktop/Gi/Supsi/MartaLauraGiada/Marta/Tweets\\2020-05-2.parquet',
 'C:/Users/gianl/Desktop/Gi/Supsi/MartaLauraGiada/Marta/Tweets\\2020-05-3.parquet']

In [6]:
original = pd.DataFrame()
count_o = 0
reply = pd.DataFrame()
count_r = 0
retweet = pd.DataFrame()
count_rt = 0
quote = pd.DataFrame()
count_q = 0
for i in tqdm(files):
    df = pd.read_parquet(i, engine="fastparquet")
    original = original.append(df[df["in_reply_to_screen_name"].isna() & df["rt_created_at"].isna() & df["quoted_status_id"].isna()])
    reply = reply.append(df[df["in_reply_to_user_id"].notna() & df["quoted_status_id"].isna()])
    retweet = retweet.append(df[df["rt_created_at"].notna()])
    quote = quote.append(df[df["quoted_status_id"].notna() & df["rt_created_at"].isna()])
    if len(original) >= int(5e6):
        original.to_parquet(path + f"original_{count_o}.parquet", index=False)
        original = pd.DataFrame()
        count_o = count_o + 1
    if len(reply) >= int(5e6):
        reply.to_parquet(path + f"reply_{count_r}.parquet", index=False)
        reply = pd.DataFrame()
        count_r = count_r + 1
    if len(retweet) >= int(5e6):
        retweet.to_parquet(path + f"retweet_{count_rt}.parquet", index=False)
        retweet = pd.DataFrame()
        count_rt = count_rt + 1
    if len(quote) >= int(5e6):
        quote.to_parquet(path + f"quote{count_q}.parquet", index=False)
        quote = pd.DataFrame()
        count_q = count_q + 1
original.to_parquet(path + f"original_{count_o}.parquet", index=False)
reply.to_parquet(path + f"reply_{count_r}.parquet", index=False)
retweet.to_parquet(path + f"retweet_{count_rt}.parquet", index=False)
quote.to_parquet(path + f"quote{count_q}.parquet", index=False)        

In [ ]:
original = 0
retweet = 0
quote = 0
reply = 0
media = pd.DataFrame()
cont = 0
for i in tqdm(files):
    cont = cont + 1
    df = pd.read_parquet(i, engine="fastparquet")
    media = media.append(df.loc[(df['urls'] != '[]') & 
                             (df['urls'] is not None) & 
                             ("https://twitter.com/" not in df['urls'])][["user_screen_name",
                                                                          "created_at", "urls"]])
    print(len(media))
    if cont == 5:
        media.to_csv(f"C:/Users/gianl/Desktop/Gi/Supsi/MartaLauraGiada/Marta/Tweets/urls_{cont}.csv",
            line_terminator="\n",
            encoding="utf-8", 
            index=False)
        media = pd.DataFrame()

In [ ]:
media.to_csv(f"C:/Users/gianl/Desktop/Gi/Supsi/MartaLauraGiada/Marta/Tweets/urls_1.csv",
            line_terminator="\n",
            encoding="utf-8", 
            index=False)

In [ ]:
media.info()

### General Info
<a id="gi"/>

In [ ]:
total = original + reply + retweet + quote
print(f"Number of total tweets: {total}")
print(f"Number of original tweets: {original}")
print(f"Number of reply: {reply}")
print(f"Number of retweet: {retweet}")
print(f"Number of quote: {quote}")

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111)
ax.bar(np.arange(4), np.asarray([original, retweet, reply, quote])*(100/total))
ax.set_xticks(np.arange(4))
ax.set_xticklabels(['tweets', 'retweets', 'replies', 'quotes'])
ax.set_ylabel('%')
ax.yaxis.grid(True)
ax.set_ylim([0,100])
ax.set_title('Distribution of Tweets')
#plt.savefig('files/plots/tweets_division')
plt.show()

## Urls
<a id="urls"/>

In [4]:
path = "C:/Users/gianl/Desktop/Gi/Supsi/MartaLauraGiada/Marta/Files/"
print("Reading 1: ...")
urls_0 = pd.read_csv(path + "urls_0.csv",lineterminator="\n",low_memory=False)
print("Reading 2: ...")
urls_1 = pd.read_csv(path + "urls_1.csv",lineterminator="\n",low_memory=False)
print("Done!")

Reading 1: ...
Reading 2: ...
Done!


In [7]:
urls = pd.concat([urls_0, urls_1])
# urls["domains"] = value

In [8]:
value = []
for i in tqdm(urls["urls"]):
    try:
        url_exp = i.split(" ")
        lst_inside = []
        for exp in range(len(url_exp)):
            if url_exp[exp] == "'expanded_url':":
                lst_inside.append(url_exp[exp+1][1:-2])
        value.append(lst_inside)
    except:
        value.append("[]")

In [9]:
def extract_domain_list(url_list):
    domain_list = []
    cont = 0
    for url in tqdm(url_list):
        cont = cont + 1
        inside = []
        if url == "[]":
            domain_list.append("[]")
        else:
            for i in url:
                try:
                    x = i.split("/")[2]
                except:
                    x = "[]"
                if "www." in x:
                    x = x[4:]
                inside.append(x)
            domain_list.append(inside)
    return domain_list

In [10]:
x = extract_domain_list(value)

In [20]:
lst = []
for i in tqdm(x):
    lst.append(len(i))

In [21]:
urls["domains"] = x
urls["number_of_url"] = lst

In [24]:
urls.to_csv("C:/Users/gianl/Desktop/Gi/Supsi/MartaLauraGiada/Marta/Files/urls_expanded.csv", 
            index=False, line_terminator="\n", encoding="utf-8")

## Domain Credibility
<a id="cred"/>

In [3]:
lst_df = []
for chunk in tqdm(pd.read_csv("C:/Users/gianl/Desktop/Gi/Supsi/MartaLauraGiada/Marta/Files/urls_expanded.csv", 
                  lineterminator="\n", low_memory=False, encoding="utf-8", chunksize=int(5e6))):
    lst_df.append(chunk)

In [5]:
path_cred = "C:/Users/gianl/Desktop/Gi/Supsi/MartaLauraGiada/"

In [6]:
wider_cred_sources_df = pd.read_csv(path_cred+'final_urls.csv')
high_cred_df = wider_cred_sources_df[wider_cred_sources_df['credibility'] == 'high']
high_cred_df.drop_duplicates(inplace=True)
low_cred_df = wider_cred_sources_df[wider_cred_sources_df['credibility'] == 'low']
low_cred_df.drop_duplicates(inplace=True)

In [7]:
social_media_df = pd.read_excel(path_cred+'social_media_sources.xlsx')
social_media_df.drop_duplicates(inplace=True)

In [10]:
low_cred_df = low_cred_df.rename(columns={"url": "LOW"})
high_cred_df = high_cred_df.rename(columns={"url": "HIGH"})

In [14]:
low = list(low_cred_df["LOW"])
high = list(high_cred_df["HIGH"])
social = list(social_media_df["SOCIAL-MEDIA"])
val = 0
for tweets_with_urls in lst_df:
    list_of_low = []
    list_of_high = []
    list_of_unkw = []
    list_of_social = []
    print(f"Iterazione {val}")
    for row in tqdm(tweets_with_urls.itertuples()):
        value_low = 0
        value_high = 0
        value_unk= 0
        value_social = 0
        for domain in row.domains:
            if domain in low:
                value_low = value_low + 1 
            elif domain in high:
                value_high = value_high + 1
            elif domain in social:
                value_social = value_social + 1
            else:
                value_unk = value_unk+ 1
        list_of_low.append(value_low)
        list_of_high.append(value_high)
        list_of_unkw.append(value_unk)
        list_of_social.append(value_social)
    lst_df[val]['LOW'] = list_of_low
    lst_df[val]['HIGH'] = list_of_high
    lst_df[val]['UNKW'] = list_of_unkw
    lst_df[val]['SOCIAL'] = list_of_social
    val = val + 1


Iterazione 0



Iterazione 1



Iterazione 2



Iterazione 3


In [ ]:
tweets_with_urls['LOW'] = list_of_low
tweets_with_urls['HIGH'] = list_of_high
tweets_with_urls['UNKW'] = list_of_unkw
tweets_with_urls['SOCIAL'] = list_of_social

In [ ]:
tweets_with_urls[tweets_with_urls['number_of_urls']>8][['number_of_urls','HIGH','LOW','UNKW','SOCIAL']]

In [ ]:
# distribution of scores
plt.hist(list(sum_of_urls_df_10['conspiracy_score']), bins=9, alpha=0.7)
plt.xlabel('conspiracy_score')
plt.ylabel('No. of users')
plt.title('Distribution of conspiracy_score', weight='bold')
#plt.savefig('plots/dist', bbox_inches='tight')
plt.show()

## Coordinated accounts
<a id="coord"/>

In [ ]:
retweets['created_at'] = pd.to_datetime(retweets['created_at'], format='%a %b %d %H:%M:%S %z %Y')
retweets['rt_created_at'] = pd.to_datetime(retweets['rt_created_at'], format='%a %b %d %H:%M:%S %z %Y')

In [ ]:
retweets['seconds_retweet'] = (retweets['rt_created_at']-retweets['created_at']).astype('timedelta64[s]')

In [ ]:
#to delete
consecutive_retweets = retweets

In [ ]:
#filter on retweet where rt_created_at-created_at<=10
seconds = 10
consecutive_retweets = retweets[retweets['seconds_retweet']<=seconds]

In [ ]:
consecutive_retweets[['user_screen_name','rt_user_screen_name']].head(10)

In [ ]:
retweets['rt_user_screen_name'] = retweets['rt_user_screen_name'].str.lower()

In [ ]:
#count number of retweets of the same user made by the another user and maintain only where count>=2
retweets_threshold = 2
consecutive_retweets_count = consecutive_retweets.groupby(['user_screen_name','rt_user_screen_name']).size().reset_index(name='counts')
consecutive_retweets_count = consecutive_retweets_count[consecutive_retweets_count['counts']>=retweets_threshold]
consecutive_retweets_count.sort_values(by='counts', ascending=False)

In [ ]:
consecutive_retweets_graph = nx.from_pandas_edgelist(consecutive_retweets_count,'user_screen_name','rt_user_screen_name', create_using=nx.DiGraph(), edge_attr='counts')
print(nx.info(consecutive_retweets_graph))

In [ ]:
isolated_nodes = list(nx.isolates(consecutive_retweets_graph))
print('Isoleted nodes:',len(isolated_nodes))

In [ ]:
nx.set_node_attributes(consecutive_retweets_graph, score_dict, "c_score")

In [ ]:
consecutive_retweets_count['rt_user_screen_name'] = consecutive_retweets_count['rt_user_screen_name'].str.lower()

In [ ]:
print('Strongly connected: ',nx.is_strongly_connected(consecutive_retweets_graph))

In [ ]:
print('Number of connected components: ',nx.number_strongly_connected_components(consecutive_retweets_graph))
print('Number of non connected components: ',len(nx.nodes(consecutive_retweets_graph))-nx.number_strongly_connected_components(consecutive_retweets_graph))

In [ ]:
# Extract the giant connected component
Gcc = sorted((consecutive_retweets_graph.subgraph(c) for c in nx.strongly_connected_components(consecutive_retweets_graph)), key = len,reverse=True)[0]
print("Number of nodes in the Gcc graph: ")
print(len(Gcc.nodes()))
print("Number of edges in the Gcc graph: ")
print(len(Gcc.edges()))

In [ ]:
# # plot the scale free graph
G = Gcc
pos = nx.drawing.layout.circular_layout(G) # compute graph layout
plt.figure(figsize=(10, 10)) 
plt.axis('off')
nx.draw_networkx_nodes(G, pos, node_size=600,node_color="green",alpha=0.3)
nx.draw_networkx_edges(G, pos, alpha=1)
nx.draw_networkx_labels(G,pos)
plt.show(G)

In [ ]:
self_retweet = consecutive_retweets_count[consecutive_retweets_count['user_screen_name']==consecutive_retweets_count['rt_user_screen_name']]
self_retweet.sort_values(by='counts', ascending=False)

In [ ]:
self_retweet_graph = nx.from_pandas_edgelist(self_retweet,'user_screen_name','rt_user_screen_name', create_using=nx.DiGraph(), edge_attr='counts')
print(nx.info(self_retweet_graph))

In [ ]:
consecutive_retweets_graph.remove_edges_from(nx.selfloop_edges(consecutive_retweets_graph))

In [ ]:
k_core = nx.k_core(consecutive_retweets_graph, k=None, core_number=None)
print(nx.info(k_core))

#### Urls to tweets network

In [ ]:
tweets_with_urls_twitter.head()

In [ ]:
tweets_with_urls_twitter = tweets_with_urls_twitter[tweets_with_urls_twitter['twitter_username']!='i']
tweets_with_urls_twitter.head()

In [ ]:
tweets_with_urls_twitter['twitter_username'] = tweets_with_urls_twitter['twitter_username'].str.lower()

In [ ]:
url_to_twitter_graph = nx.from_pandas_edgelist(tweets_with_urls_twitter, 'user_screen_name', 'twitter_username',create_using=nx.DiGraph())
print(nx.info(url_to_twitter_graph))

In [ ]:
degrees = [val for (node, val) in url_to_twitter_graph.degree()]

print(f"The maximum degree of the Graph is {np.max(degrees)}")   
print(f"The minimum degree of the Graph is {np.min(degrees)}")

out_degrees = [val for (node, val) in url_to_twitter_graph.out_degree()]

print(f"The maximum out_degree of the Graph is {np.max(out_degrees)}")   
print(f"The minimum out_degree of the Graph is {np.min(out_degrees)}")

in_degrees = [val for (node, val) in url_to_twitter_graph.in_degree()]

print(f"The maximum in_degree of the Graph is {np.max(in_degrees)}")   
print(f"The minimum in_degree of the Graph is {np.min(in_degrees)}")

In [ ]:
url_to_twitter_indegree_list = list(url_to_twitter_graph.in_degree(url_to_twitter_graph.nodes()))
df_indegree = pd.DataFrame(url_to_twitter_indegree_list, columns=['user_screen_name','url_to_twitter_in_degree'])
users_info = users_info.merge(df_indegree, how='left',on='user_screen_name')

In [ ]:
url_to_twitter_outdegree_list = list(url_to_twitter_graph.out_degree(url_to_twitter_graph.nodes()))
df_outdegree = pd.DataFrame(url_to_twitter_outdegree_list, columns=['user_screen_name','url_to_twitter_out_degree'])
users_info = users_info.merge(df_outdegree, how='left',on='user_screen_name')

In [ ]:
tweets_with_hashtags['max_hashtags']=tweets_with_hashtags.groupby(['user_screen_name'])['number_of_hashtags'].transform(max)

In [ ]:
hashtags_df = tweets_with_hashtags.drop_duplicates(['user_screen_name'])[['user_screen_name','max_hashtags']]

In [ ]:
users_info = users_info.merge(hashtags_df, how='left',on='user_screen_name')

In [ ]:
def get_set_of_hashtags_per_user(hashtag_list):
    set_of_hashtags = list(set([item for sublist in hashtag_list for item in sublist]))
    return set_of_hashtags

In [ ]:
func = tweets_with_hashtags.groupby("user_screen_name").apply(lambda x: get_set_of_hashtags_per_user(x.hashtag_list))
users_info['set_hashtags'] = users_info['user_screen_name'].map(func)
##

In [ ]:
users_info.head()

In [ ]:
users_many_hashtags = users_info[users_info['max_hashtags']>20]

In [ ]:
hashtags_users_many_hashtags = users_many_hashtags['set_hashtags']
list_hashtags_users_many_hashtags = [item for sublist in hashtags_users_many_hashtags for item in sublist]

In [ ]:
from collections import Counter
count_common_hashtags = Counter(list_hashtags_users_many_hashtags)
count_common_hashtags_sort = dict(sorted(count_common_hashtags.items(), key=lambda item: item[1], reverse=True))

In [ ]:
len(count_common_hashtags_sort)

In [ ]:
plt.subplots(figsize=(15,10))
plt.barh(list(count_common_hashtags_sort.keys())[:50], list(count_common_hashtags_sort.values())[:50], 2, color='g')

In [ ]:
tweets_with_urls['max_urls']=tweets_with_urls.groupby(['user_screen_name'])['number_of_urls'].transform(max)
urls_df = tweets_with_urls.drop_duplicates(['user_screen_name'])[['user_screen_name','max_urls']]
users_info = users_info.merge(urls_df, how='left',on='user_screen_name')

In [ ]:
users_info.info(null_counts=True)

In [ ]:
corr = users_info.corr()
corr

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111)
sns.heatmap(corr, annot=True, ax=ax)

In [ ]:
users_info.sort_values(by='followers_count',ascending=False)[:20]

In [ ]:
users_info_cosp_0 = users_info[users_info['conspiracy_score']==0]
users_info_cosp_1 = users_info[users_info['conspiracy_score']>0]
print('Number of users conspiracy_score==0: ',len(users_info_cosp_0[users_info_cosp_0['followers_count']<=30000]))
print('Number of users conspiracy_score>0: ',len(users_info_cosp_1[users_info_cosp_1['followers_count']<=30000]))

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 7)) 

axes[0].hist(users_info_cosp_0[users_info_cosp_0['followers_count']<=30000]['followers_count'], bins=50)
axes[1].hist(users_info_cosp_1[users_info_cosp_1['followers_count']<=30000]['followers_count'], bins=25)

#axes[0].set_xlim(0,200000)
#axes[1].set_xlim(0,40000)

axes[0].set_xlabel('Followers')
axes[1].set_xlabel('Followers')
axes[0].set_ylabel('Count')
axes[1].set_ylabel('Count')
axes[0].set_title('followers conspiracy_score==0')
axes[1].set_title('followers conspiracy_score>0')
axes[0].set_xlim(0,30000)
axes[1].set_xlim(0,30000)
axes[0].set_ylim(0,150)
axes[1].set_ylim(0,150)
#plt.ylabel('F1score')
#plt.ylim([0, 1])
#plt.title('Low resolution dataset: F1-score lrt=1e-4')
#plt.savefig(path_plot+'f1score_lrt=1e-4_all_classes', bbox_inches='tight')
plt.show()

In [ ]:
users_info_cosp_0 = users_info[users_info['conspiracy_score']==0]
users_info_cosp_1 = users_info[users_info['conspiracy_score']>0]
print(len(users_info_cosp_0[users_info_cosp_0['retweets_in_degree'].notna()]))
print(len(users_info_cosp_1[users_info_cosp_1['retweets_in_degree'].notna()]))

In [ ]:
print(len(users_info_cosp_0[users_info_cosp_0['retweets_in_degree']<=1000]))
print(len(users_info_cosp_1[users_info_cosp_1['retweets_in_degree']<=1000]))

In [ ]:
users_info_cosp_0 = users_info[users_info['conspiracy_score']==0]
users_info_cosp_1 = users_info[users_info['conspiracy_score']>0]
print('Number of users conspiracy_score==0: ',len(users_info_cosp_0[users_info_cosp_0['retweets_in_degree']<=1000]))
print('Number of users conspiracy_score>0: ',len(users_info_cosp_1[users_info_cosp_1['retweets_in_degree']<=1000]))

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 7)) 

axes[0].hist(users_info_cosp_0[users_info_cosp_0['retweets_in_degree']<=1000]['retweets_in_degree'], bins=100)
axes[1].hist(users_info_cosp_1[users_info_cosp_1['retweets_in_degree']<=1000]['retweets_in_degree'], bins=50)

#axes[0].set_xlim(0,200000)
#axes[1].set_xlim(0,40000)

axes[0].set_xlabel('retweets_in_degree')
axes[1].set_xlabel('retweets_in_degree')
axes[0].set_ylabel('Count')
axes[1].set_ylabel('Count')
axes[0].set_title('retweets_in_degree conspiracy_score==0')
axes[1].set_title('retweets_in_degree conspiracy_score>0')
axes[0].set_xlim(0,1000)
axes[1].set_xlim(0,1000)
axes[0].set_ylim(0,600)
axes[1].set_ylim(0,600)
#plt.ylabel('F1score')
#plt.ylim([0, 1])
#plt.title('Low resolution dataset: F1-score lrt=1e-4')
#plt.savefig(path_plot+'f1score_lrt=1e-4_all_classes', bbox_inches='tight')
plt.show()

In [ ]:
print(len(users_info_cosp_0[users_info_cosp_0['retweets_out_degree'].notna()]))
print(len(users_info_cosp_1[users_info_cosp_1['retweets_out_degree'].notna()]))

In [ ]:
print(len(users_info_cosp_0[users_info_cosp_0['retweets_out_degree']<=250]))
print(len(users_info_cosp_1[users_info_cosp_1['retweets_out_degree']<=250]))

In [ ]:
users_info_cosp_0 = users_info[users_info['conspiracy_score']==0]
users_info_cosp_1 = users_info[users_info['conspiracy_score']>0]
print('Number of users conspiracy_score==0: ',len(users_info_cosp_0[users_info_cosp_0['retweets_out_degree']<=250]))
print('Number of users conspiracy_score>0.5: ',len(users_info_cosp_1[users_info_cosp_1['retweets_out_degree']<=250]))

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 7)) 

axes[0].hist(users_info_cosp_0[users_info_cosp_0['retweets_out_degree']<=250]['retweets_out_degree'], bins=100)
axes[1].hist(users_info_cosp_1[users_info_cosp_1['retweets_out_degree']<=250]['retweets_out_degree'], bins=50)

#axes[0].set_xlim(0,200000)
#axes[1].set_xlim(0,40000)

axes[0].set_xlabel('retweets_out_degree')
axes[1].set_xlabel('retweets_out_degree')
axes[0].set_ylabel('Count')
axes[1].set_ylabel('Count')
axes[0].set_title('retweets_out_degree conspiracy_score==0')
axes[1].set_title('retweets_out_degree conspiracy_score>0')
axes[0].set_xlim(0,250)
axes[1].set_xlim(0,250)
axes[0].set_ylim(0,250)
axes[1].set_ylim(0,250)
#plt.ylabel('F1score')
#plt.ylim([0, 1])
#plt.title('Low resolution dataset: F1-score lrt=1e-4')
#plt.savefig(path_plot+'f1score_lrt=1e-4_all_classes', bbox_inches='tight')
plt.show()

In [ ]:
print(len(users_info_cosp_0[users_info_cosp_0['number_of_retweets'].notna()]))
print(len(users_info_cosp_1[users_info_cosp_1['number_of_retweets'].notna()]))

In [ ]:
print(len(users_info_cosp_0[users_info_cosp_0['number_of_retweets']<=500]))
print(len(users_info_cosp_1[users_info_cosp_1['number_of_retweets']<=500]))

In [ ]:
users_info_cosp_0 = users_info[users_info['conspiracy_score']==0]
users_info_cosp_1 = users_info[users_info['conspiracy_score']>0]
print('Number of users conspiracy_score==0: ',len(users_info_cosp_0[users_info_cosp_0['number_of_retweets']<=500]))
print('Number of users conspiracy_score>0: ',len(users_info_cosp_1[users_info_cosp_1['number_of_retweets']<=500]))

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 7)) 

axes[0].hist(users_info_cosp_0[users_info_cosp_0['number_of_retweets']<=500]['number_of_retweets'], bins=100)
axes[1].hist(users_info_cosp_1[users_info_cosp_1['number_of_retweets']<=500]['number_of_retweets'], bins=50)

#axes[0].set_xlim(0,200000)
#axes[1].set_xlim(0,40000)

axes[0].set_xlabel('number_of_retweets')
axes[1].set_xlabel('number_of_retweets')
axes[0].set_ylabel('Count')
axes[1].set_ylabel('Count')
axes[0].set_title('number_of_retweets conspiracy_score==0')
axes[1].set_title('number_of_retweets conspiracy_score>0')
axes[0].set_xlim(0,500)
axes[1].set_xlim(0,500)
axes[0].set_ylim(0,125)
axes[1].set_ylim(0,125)
#plt.ylabel('F1score')
#plt.ylim([0, 1])
#plt.title('Low resolution dataset: F1-score lrt=1e-4')
#plt.savefig(path_plot+'f1score_lrt=1e-4_all_classes', bbox_inches='tight')
plt.show()